In [ ]:

from flask import Flask, render_template, request, redirect
import boto3
from cryptography.fernet import Fernet

app = Flask(_name_)

# Initialize S3 client with correct region and signature version
s3 = boto3.client(
    's3',
    region_name='eu-north-1',  # Use your actual region
    config=boto3.session.Config(signature_version='s3v4')
)

BUCKET_NAME = 'my-image-management-bucket'  # Replace with your S3 bucket name

# Generate an encryption key (This should be securely stored in a real-world application)
encryption_key = Fernet.generate_key()
cipher = Fernet(encryption_key)

# Route for displaying the upload form and uploading images
@app.route('/')
def index():
    return render_template('index.html')

# Route for uploading images
@app.route('/upload', methods=['POST'])
def upload_image():
    if 'file' not in request.files:
        return 'No file part'
    file = request.files['file']
    if file.filename == '':
        return 'No selected file'
    
    # Check if the user wants to encrypt the image
    if 'encrypt' in request.form:
        file_data = file.read()  # Read the file data
        encrypted_data = cipher.encrypt(file_data)  # Encrypt the data
        s3.put_object(Bucket=BUCKET_NAME, Key=file.filename + '.encrypted', Body=encrypted_data)
        return 'Image encrypted and uploaded successfully!'
    else:
        s3.upload_fileobj(file, BUCKET_NAME, file.filename)
        return 'Image uploaded successfully!'

# Route for listing images stored in the S3 bucket
@app.route('/list', methods=['GET'])
def list_images():
    objects = s3.list_objects_v2(Bucket=BUCKET_NAME)

    files = []
    if 'Contents' in objects:
        for obj in objects['Contents']:
            if 'encrypted' in request.args and request.args['encrypted'] == 'on':
                # If the "Show only encrypted images" checkbox is checked, filter files accordingly
                if obj['Key'].endswith('.encrypted'):  # Assuming encrypted files have a specific extension
                    files.append(obj['Key'])
            else:
                files.append(obj['Key'])
                
    return render_template('list.html', files=files)

# Route for downloading an image
@app.route('/download/<filename>')
def download_image(filename):
    # Generate a presigned URL to download the file from S3
    file_url = s3.generate_presigned_url('get_object',
                                         Params={'Bucket': BUCKET_NAME, 'Key': filename},
                                         ExpiresIn=3600)  # Link valid for 1 hour
    return redirect(file_url)

if _name_ == '_main_':
    app.run(debug=True, host='0.0.0.0')